# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

weatherpy_csv = "output_data_file"

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacationpy_df = pd.read_csv(weatherpy_csv)
vacationpy_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,hervey bay,-25.2986,152.8535,73.00,63,52,8.01,AU,1619495977
1,belmonte,-15.8631,-38.8828,75.20,78,20,5.75,BR,1619495978
2,punta arenas,-53.1500,-70.9167,44.60,93,90,5.75,CL,1619495978
3,geraldton,-28.7667,114.6000,87.80,20,20,3.44,AU,1619495979
4,oranjemund,-28.5500,16.4333,59.04,86,79,5.12,NaN,1619495979


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
locations = vacationpy_df[["Lat", "Lng"]]
hum = vacationpy_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=hum, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
### Create new DataFrame fitting weather criteria
# * Narrow down the cities to fit weather conditions.
# * Drop any rows will null values.

In [6]:
vacationpy_weather_crit = vacationpy_df[vacationpy_df['Max Temp'] < 80] 
vacationpy_weather_crit = vacationpy_weather_crit[vacationpy_df['Max Temp'] > 70]
vacationpy_weather_crit = vacationpy_weather_crit[vacationpy_df['Wind Speed'] < 10]
vacationpy_weather_crit = vacationpy_weather_crit[vacationpy_df['Cloudiness'] == 0]
vacationpy_weather_crit

<ipython-input-6-1ea31caf26ac>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  vacationpy_weather_crit = vacationpy_weather_crit[vacationpy_df['Max Temp'] > 70]
<ipython-input-6-1ea31caf26ac>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  vacationpy_weather_crit = vacationpy_weather_crit[vacationpy_df['Wind Speed'] < 10]
<ipython-input-6-1ea31caf26ac>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  vacationpy_weather_crit = vacationpy_weather_crit[vacationpy_df['Cloudiness'] == 0]


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
195,lenger,42.1830,69.8839,73.4,38,0,2.24,KZ,1619496084
250,casa nova,-9.4167,-41.1333,78.8,65,0,9.22,BR,1619496107
302,meybod,32.2501,54.0166,71.6,37,0,2.30,IR,1619496128
421,conceicao do araguaia,-8.2578,-49.2647,77.0,94,0,3.44,BR,1619496177
423,riyadh,24.6877,46.7219,77.0,22,0,8.68,SA,1619496007
425,dubbo,-32.2500,148.6167,75.2,29,0,2.30,AU,1619496178
570,silvania,-16.6589,-48.6081,73.4,73,0,4.61,BR,1619496237


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
vacationpy_weather_crit['Hotel Name'] = ''
vacationpy_weather_crit

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
195,lenger,42.1830,69.8839,73.4,38,0,2.24,KZ,1619496084,
250,casa nova,-9.4167,-41.1333,78.8,65,0,9.22,BR,1619496107,
302,meybod,32.2501,54.0166,71.6,37,0,2.30,IR,1619496128,
421,conceicao do araguaia,-8.2578,-49.2647,77.0,94,0,3.44,BR,1619496177,
423,riyadh,24.6877,46.7219,77.0,22,0,8.68,SA,1619496007,
425,dubbo,-32.2500,148.6167,75.2,29,0,2.30,AU,1619496178,
570,silvania,-16.6589,-48.6081,73.4,73,0,4.61,BR,1619496237,


In [8]:
# target_coordinates = vacationpy_weather_crit[['Lat','Lng']]
# target_coordinates = target_coordinates.reset_index(drop=True)
vacationpy_weather_crit = vacationpy_weather_crit.reset_index(drop=True)
vacationpy_weather_crit

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,lenger,42.1830,69.8839,73.4,38,0,2.24,KZ,1619496084,
1,casa nova,-9.4167,-41.1333,78.8,65,0,9.22,BR,1619496107,
2,meybod,32.2501,54.0166,71.6,37,0,2.30,IR,1619496128,
3,conceicao do araguaia,-8.2578,-49.2647,77.0,94,0,3.44,BR,1619496177,
4,riyadh,24.6877,46.7219,77.0,22,0,8.68,SA,1619496007,
5,dubbo,-32.2500,148.6167,75.2,29,0,2.30,AU,1619496178,
6,silvania,-16.6589,-48.6081,73.4,73,0,4.61,BR,1619496237,


In [9]:
vacationpy_weather_crit[['Lat','Lng']]

,Lat,Lng
0,42.1830,69.8839
1,-9.4167,-41.1333
2,32.2501,54.0166
3,-8.2578,-49.2647
4,24.6877,46.7219
5,-32.2500,148.6167
6,-16.6589,-48.6081


In [10]:
hotel_df = []

for vacay in range(len(vacationpy_weather_crit)):
    target_coordinates_lat = vacationpy_weather_crit.loc[vacay]['Lat']
    target_coordinates_lng = vacationpy_weather_crit.loc[vacay]['Lng']
    target_coordinates_city = vacationpy_weather_crit.loc[vacay]['City']
    target_coordinates_country = vacationpy_weather_crit.loc[vacay]['Country']


    # geocoordinates
    target_type = "lodging"
    target_radius = 5000

    # set up a parameters dictionary
    params = {
        "location": f"{target_coordinates_lat}, {target_coordinates_lng}",
        "radius": target_radius,
        "type": target_type,
#         "City": target_coordinates_city,
#         "Country": target_coordinates_country,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    time.sleep(2)
    hotel_json = response.json()
    
    try:
        hotel_df.append(hotel_json['results'][0]['name'])

    except:
        hotel_df.append('')
    
    print(json.dumps(hotel_json, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 42.17735619999999,
                    "lng": 69.88280370000001
                },
                "viewport": {
                    "northeast": {
                        "lat": 42.17867363029151,
                        "lng": 69.88405798029152
                    },
                    "southwest": {
                        "lat": 42.17597566970851,
                        "lng": 69.88136001970851
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Gostevoy Dom 1",
            "photos": [
                {
                    "height": 2976,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/105698649747790217406\">S

{
    "html_attributions": [],
    "next_page_token": "ATtYBwIf8MM1NSKnrA2Dljh6dMvH5KpkFXLjsNcPe8cyjDCxG_UIxc4VBtY5SrA9GyTpNSTfw_8mIki177kkEVYLL7U_fFoiy3KO-T5x9V5pVKBtwhX2cvAY8RGXxUdJTPCNS2lZyC6a5P_T35OrV7EAsGsF5qMHT5wNNKKhSDphZ7av-xnXdB9QwZ73iUl5nPcyIkd76VDCOnzyQ4EMzDKunNQXD4zeds3BKPwFYGgCsz-ShZ9Q4XNpH_ikRcNwmHwJcN45-YB68gmqMC2t2ay_IFQtydK8qDd8QzkuTXv1VaUlPLv6jSP5BZDEx3roZ516pLlQHwf6DruBeqL86GMBdd0HAsUyRdxzXgiq07M_uT5hnADwd1HjmkM0FPii1fLL_Rnc0scuO8_GnpB_A7EGDw2wsGbTAoBOtqV1Z7UUlsO3yOKOLTMC4vlB",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -8.272012,
                    "lng": -49.260772
                },
                "viewport": {
                    "northeast": {
                        "lat": -8.2708450197085,
                        "lng": -49.2595345697085
                    },
                    "southwest": {
                        "lat": -8.273542980291504,
  

{
    "html_attributions": [],
    "next_page_token": "ATtYBwKE7q8JqWtjLxR-O1f8A3kHwY2KsQ2gYcZnKl4h7AxPY-HE8TsaGZZgam2cakwiw_PSvEaJ1UEvN-yG2mVJoM_1cKgK54qOLKR6A29Z1qYwwZmDgJxaKpKrqqWz-ia6p76yGPHJVpMwOhmrkMGXjukm8r5Gjxxu0WaEHwEavsqBjiRvZQNJKpzAevTuf5Ou9BCl8Ym_Ci1fQmn14bgjH0pbt7JbM7UUQQw4Q2a9jgNicbtQvd6iFu2VT1nGpegXGMrQMKu1kRm5Ou3S3ON3UFSF_qojpamB9vGnAgT66WgoAXRM1pe3phmuUQuu6BdkTs3_z9NL9bbxKB7FaMJ43gQrAwegyQFs_XLIbzSfBmZ5E1mOrptzPtnzv_mPTbUCK5SHsSmn8nH2TNZwF1iQ9GcaA8-C9p6WG7BBrSiBDv2BiN0f8yjFENvb",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 24.666677,
                    "lng": 46.692159
                },
                "viewport": {
                    "northeast": {
                        "lat": 24.6677663802915,
                        "lng": 46.6937272802915
                    },
                    "southwest": {
                        "lat": 24.6650684197085,
      

{
    "html_attributions": [],
    "next_page_token": "ATtYBwLUkTcAy-EPUKT_th2f5O7quDVs5ujPIui7ZL2-CbNhpyJVWPwoptcywht9hbwCm5e9EnaP92i7WBsZDac_OlGu9wTZvsKfSBazdrrPRIXGgg7yTng_xh6CtImw5cCn8eWl2WLQ2lheJChNmidukEOojFgrwvf6ldhoynR5cUNmUJ429zkF7cftvVmPxiVUwwPAl0cbrzfW87pNso_novPt3IlyAoZ9QTjxCNZ9AP5ahNvOuYy__JE0uN1zeVaj40nLJpdPrdnw0F5IJrxDzTgS7A1LQavdBkH2Fx9Ql-mGl8ikVRvyKSPXWcgyP7jF7daWZHWHVmTgmwOJojXy7PXwApV1DPnckr_MrrPobUbo6fv_1v3wQ-UoHu3CpktwdvOJK15xRYOfc0XzLR6_CZb31xA7lSkuxHwiITAxzo_KJX0DI7SJCtyD",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -32.25444839999999,
                    "lng": 148.6070196
                },
                "viewport": {
                    "northeast": {
                        "lat": -32.2529005197085,
                        "lng": 148.6084031302915
                    },
                    "southwest": {
                        "lat": -32.2555984

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -16.6747127,
                    "lng": -48.61670729999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -16.6734152697085,
                        "lng": -48.6152337697085
                    },
                    "southwest": {
                        "lat": -16.6761132302915,
                        "lng": -48.6179317302915
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Hotel Dom Bosco",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 960,
                    "html_attributions": [
                       

In [11]:
hotel_df

['Gostevoy Dom 1',
 'Pousada e Restaurante Dona Xinha',
 'Traditional Hotel Restaurant Haji Malek',
 'Tarumã Tropical Hotel',
 'InterContinental Riyadh, an IHG Hotel',
 'Comfort Inn Blue Lagoon',
 'Hotel Dom Bosco']

In [14]:
hotel_2_df = pd.DataFrame({'Hotel Name': hotel_df})
hotel_2_df

,Hotel Name
0,Gostevoy Dom 1
1,Pousada e Restaurante Dona Xinha
2,Traditional Hotel Restaurant Haji Malek
3,Tarumã Tropical Hotel
4,"InterContinental Riyadh, an IHG Hotel"
5,Comfort Inn Blue Lagoon
6,Hotel Dom Bosco


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_2_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'City'

In [16]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
figure.add_layer(markers)

# Display figure
figure

NameError: name 'figure' is not defined